<a href="https://colab.research.google.com/github/NapoliD/Spark/blob/main/Solucion_Properati_Spark_DataFrames_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Start a simple Spark Session

In [ ]:
pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 38.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=b65531849c043a7f6f9a56b31012ba49ef4c89c0208c33b8ae5bf9645cfd75d1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/My Drive/GOOGLE_COLAB/Spark/Properati')

In [ ]:
import pandas as pd
df = pd.read_csv ('Properati.csv')
df.to_json ('Properati.json')

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("properati").getOrCreate()

#### Load  CSV File, have Spark infer the data types.

In [ ]:
cases = spark.read.load("Properati - Properati.csv",format="csv", sep=",", inferSchema="true", header="false")
cases.show(100)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|                 _c4|                 _c5|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|"{""tracking_date...|""country_code"":...|""seller_id"":""8...|""seller_sk"":""A...|  ""counter1"":""1""|""counter2"":""1""}"|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|""counter2"":""1""}"|                null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""3...|""seller_sk"":""A...|""counter2"":""1""}"|                null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|""counter2"":""1""}"|                null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|""counter2"":""1""}

In [ ]:
cases.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



In [ ]:
cases = cases.withColumn('_c4', regexp_replace('_c4', '}', ''))
cases = cases.withColumn('_c4', regexp_replace('_c4', '"', ''))
cases = cases.withColumn('_c5', regexp_replace('_c5', '}', ''))
cases = cases.withColumn('_c5', regexp_replace('_c5', '"', ''))
cases.show()

+--------------------+--------------------+--------------------+--------------------+----------+----------+
|                 _c0|                 _c1|                 _c2|                 _c3|       _c4|       _c5|
+--------------------+--------------------+--------------------+--------------------+----------+----------+
|"{""tracking_date...|""country_code"":...|""seller_id"":""8...|""seller_sk"":""A...|counter1:1|counter2:1|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|counter2:1|      null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""3...|""seller_sk"":""A...|counter2:1|      null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|counter2:1|      null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|counter2:1|      null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""8...|""seller_sk"":""A...|counter2:2|      null|
|"{""tracking_date...|""coun

In [ ]:
### String Split of the column in pyspark

from pyspark.sql.functions import split,col

cases=cases.withColumn("tracking_date", split(col("_c0"), ":").getItem(1)).withColumn("country_code", split(col("_c1"), ":").getItem(1)).withColumn("seller_id", split(col("_c2"), ":").getItem(1)).withColumn("seller_sk", split(col("_c3"), ":").getItem(1))
cases=cases.withColumn("counter1", split(col("_c4"), "counter1:").getItem(1)).withColumn("counter2", split(col("_c4"), "counter2:").getItem(1)).withColumn("counter2a", split(col("_c5"), "counter2:").getItem(1))
#cases=cases.withColumn("counter2", split(col("_c5"), "counter2:").getItem(1))


In [ ]:
cases.show(1000)

In [ ]:
### Remove leading space of the column in pyspark
from pyspark.sql.functions import *
#df2 = df.withColumn('_c0', regexp_replace('_c0', 'tracking_date', ''))
df2 = cases.withColumn('tracking_date', regexp_replace('tracking_date', '"', ''))
#df2 = df2.withColumn('_c0', regexp_replace('_c0', ':', ''))

#df2 = df2.withColumn('_c1', regexp_replace('_c1', 'country_code', ''))
df2 = df2.withColumn('country_code', regexp_replace('country_code', '"', ''))
#df2 = df2.withColumn('_c1', regexp_replace('_c1', ':', ''))

#df2 = df2.withColumn('_c2', regexp_replace('_c2', 'seller_id', ''))
df2 = df2.withColumn('seller_id', regexp_replace('seller_id', '"', ''))
#df2 = df2.withColumn('_c2', regexp_replace('_c2', ':', ''))

#df2 = df2.withColumn('_c3', regexp_replace('_c3', 'seller_sk', ''))
df2 = df2.withColumn('seller_sk', regexp_replace('seller_sk', '"', ''))
#df2 = df2.withColumn('_c3', regexp_replace('_c3', ':', ''))

df2 = df2.withColumn('counter1', regexp_replace('counter1', '"}', ''))
df2 = df2.withColumn('counter2', regexp_replace('counter2', '"}', ''))

df2 = df2.withColumn('counter1', regexp_replace('counter1', '"', ''))
df2 = df2.withColumn('counter2', regexp_replace('counter2', '"', ''))
#df2
#df2

df2.show()


+--------------------+--------------------+--------------------+--------------------+----------+----------+-------------+------------+---------+---------+--------+--------+---------+
|                 _c0|                 _c1|                 _c2|                 _c3|       _c4|       _c5|tracking_date|country_code|seller_id|seller_sk|counter1|counter2|counter2a|
+--------------------+--------------------+--------------------+--------------------+----------+----------+-------------+------------+---------+---------+--------+--------+---------+
|"{""tracking_date...|""country_code"":...|""seller_id"":""8...|""seller_sk"":""A...|counter1:1|counter2:1|   2020-10-07|          AR|   820417|AR|820417|       1|    null|        1|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|counter2:1|      null|   2020-10-07|          AR|    23899| AR|23899|    null|       1|     null|
|"{""tracking_date...|""country_code"":...|""seller_id"":""3...|""seller_sk"":""A...|

In [ ]:
df2 = df2.select('tracking_date','counter1','counter2','counter2a')
df2.show()

+-------------+--------+--------+---------+
|tracking_date|counter1|counter2|counter2a|
+-------------+--------+--------+---------+
|   2020-10-07|       1|    null|        1|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       2|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       2|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       2|     null|
|   2020-10-07|    null|       2|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1|     null|
|   2020-10-07|    null|       1

In [ ]:
from pyspark.sql.types import IntegerType,TimestampType
df2 = df2.withColumn("tracking_date", df2["tracking_date"].cast(TimestampType()))
df2 = df2.withColumn("counter1", df2["counter1"].cast(IntegerType()))
df2 = df2.withColumn("counter2", df2["counter2"].cast(IntegerType()))

df2 = df2.withColumn("counter2a", df2["counter2a"].cast(IntegerType()))
df2.show()

+-------------------+--------+--------+---------+
|      tracking_date|counter1|counter2|counter2a|
+-------------------+--------+--------+---------+
|2020-10-07 00:00:00|       1|    null|        1|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       2|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       2|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       1|     null|
|2020-10-07 00:00:00|    null|       2|     null|
|2020-10-07 00:00:00|    null|       2|     null|
|2020-10-07 00:00:00|    null|       1|     null|


In [ ]:
df2 = df2.withColumn(
    'counter2',
    F.when((F.col("counter2") == 'null'), F.col("counter2a"),).otherwise(F.col("counter2a"))
)

df2.show()

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
def somefunc(value):
  if   value < 3: 
      return 'low'
  else:
      return 'high'
#convert to a UDF Function by passing in the function and return type of function
udfsomefunc = F.udf(somefunc, StringType())
ratings_with_high_low = ratings.withColumn("high_low", udfsomefunc("rating"))
ratings_with_high_low.show()

In [ ]:
df2.printSchema()

root
 |-- tracking_date: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- seller_sk: string (nullable = true)
 |-- counter1: string (nullable = true)
 |-- counter2: string (nullable = true)



In [ ]:
from pyspark.sql.types import IntegerType,TimestampType
df2 = df2.withColumn("tracking_date", df2["tracking_date"].cast(TimestampType()))
df2 = df2.withColumn("seller_id", df2["seller_id"].cast(IntegerType()))
df2 = df2.withColumn("seller_sk", df2["seller_sk"].cast(IntegerType()))

df2 = df2.withColumn("seller_sk", df2["counter1"].cast(IntegerType()))
df2 = df2.withColumn("seller_sk", df2["counter1"].cast(IntegerType()))
df2.show()

+-------------------+------------+---------+---------+--------+--------+
|      tracking_date|country_code|seller_id|seller_sk|counter1|counter2|
+-------------------+------------+---------+---------+--------+--------+
|2020-10-07 00:00:00|          AR|   820417|        1|       1|      1}|
|2020-10-07 00:00:00|          AR|    23899|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|     3178|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|    22247|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|     2924|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|   820086|     null|      2}|    null|
|2020-10-07 00:00:00|          AR|    23323|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|    28121|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|    20738|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|   673681|     null|      1}|    null|
|2020-10-07 00:00:00|          AR|   692032|     nu

In [ ]:
df2.printSchema()

root
 |-- tracking_date: timestamp (nullable = true)
 |-- country_code: string (nullable = true)
 |-- seller_id: integer (nullable = true)
 |-- seller_sk: integer (nullable = true)



In [ ]:
df2.write.csv('final.csv')

In [ ]:
cases = spark.read.load("final.csv",format="csv", sep=",", inferSchema="true", header="false")
cases.show()

+-------------------+---+------+------+
|                _c0|_c1|   _c2|   _c3|
+-------------------+---+------+------+
|2020-10-07 00:00:00| AR| 26606| 26606|
|2020-10-07 00:00:00| AR| 23899| 23899|
|2020-10-07 00:00:00| AR|  3178|  3178|
|2020-10-07 00:00:00| AR| 22247| 22247|
|2020-10-07 00:00:00| AR|  2924|  2924|
|2020-10-07 00:00:00| AR|820086|820086|
|2020-10-07 00:00:00| AR| 24914| 24914|
+-------------------+---+------+------+



In [ ]:
df2.createOrRemplaceTempView('people')

AttributeError: ignored

In [ ]:
sql_results = spark.sql("SELECT * FROM people")

In [ ]:
sql_results

In [ ]:
df2.toPandas().to_csv('Data_pandas.csv')

In [ ]:
import pandas as pd
df2_pandas=df2.toPandas()
df2_pandas=pd.DataFrame(df2_pandas)
df2_pandas.dtypes

tracking_date    datetime64[ns]
country_code             object
seller_id                 int32
seller_sk                 int32
dtype: object

In [ ]:
df2_pandas

,tracking_date,country_code,seller_id,seller_sk
0,2020-10-07,AR,26606,26606
1,2020-10-07,AR,23899,23899
2,2020-10-07,AR,3178,3178
3,2020-10-07,AR,22247,22247
4,2020-10-07,AR,2924,2924
5,2020-10-07,AR,820086,820086
6,2020-10-07,AR,24914,24914


https://towardsdatascience.com/the-most-complete-guide-to-pyspark-dataframes-2702c343b2e8#844d

In [ ]:
def lower_clean_str(x):
  punc='"'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str


In [ ]:
cases_pandas['_c0']=cases_pandas['_c0'].map(lower_clean_str)
cases_pandas['_c1']=cases_pandas['_c1'].map(lower_clean_str)
cases_pandas['_c2']=cases_pandas['_c2'].map(lower_clean_str)
cases_pandas['_c3']=cases_pandas['_c3'].map(lower_clean_str)

In [ ]:
df.show

In [ ]:
cases.write.csv('mycsv.csv')

In [ ]:
df=cases.toPandas()

In [ ]:
df.describe()

,tracking_date,country_code,seller_id,seller_sk
count,7,7,7,7
unique,1,1,7,7
top,"""""2020-10-07""""","""""AR""""","""""820086""""","""""AR|2924"""""
freq,7,7,1,1


In [ ]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,TimestampType

In [ ]:
data_schema = [StructField("tracking_date", TimestampType(),True),StructField("seller_id", IntegerType(), True),StructField("seller_sk", IntegerType(), True)]

In [ ]:
final_struc = StructType(fields=data_schema)

Change columns

In [ ]:
#import pyspark.sql.functions as f
#tracking_date = cases.select(f.split(cases.tracking_date,":")).rdd.flatMap(lambda x: x).toDF(schema=["col1","tracking_date"])
#tracking_date.show()
#df = cases.withColumn("tracking_date2", cases.tracking_date.substr(25,8))
#df = df.withColumn("country_code2", df.country_code.substr(20,2))
#df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+-------------+
|       tracking_date|        country_code|           seller_id|           seller_sk|tracking_date2|country_code2|
+--------------------+--------------------+--------------------+--------------------+--------------+-------------+
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|      20-10-07|           AR|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|      20-10-07|           AR|
|"{""tracking_date...|""country_code"":...|""seller_id"":""3...|""seller_sk"":""A...|      20-10-07|           AR|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|      20-10-07|           AR|
|"{""tracking_date...|""country_code"":...|""seller_id"":""2...|""seller_sk"":""A...|      20-10-07|           AR|
|"{""tracking_date...|""country_code"":...|""seller_id"":""8...|""seller_sk"":""

In [ ]:
df = spark.read.json("Properati_3.json",multiLine = "true")
df.printSchema()

root
 |-- {"tracking_date":"2020-10-07","country_code":"AR","seller_id":"26606","seller_sk":"AR|26606","counter2":"1"}: struct (nullable = true)
 |    |-- 0: string (nullable = true)
 |    |-- 1: string (nullable = true)
 |    |-- 2: string (nullable = true)
 |    |-- 3: string (nullable = true)
 |    |-- 4: string (nullable = true)



In [ ]:
sampleDF = df.withColumnRenamed ("2", "clave")
sampleDF

DataFrame[{"tracking_date":"2020-10-07","country_code":"AR","seller_id":"26606","seller_sk":"AR|26606","counter2":"1"}: struct<0:string,1:string,2:string,3:string,4:string>]

In [ ]:
batDF = sampleDF.select ("clave", "seller_id") 
batDF.printSchema ()

https://kontext.tech/column/spark/284/pyspark-convert-json-string-column-to-array-of-object-structtype-in-data-frame

In [ ]:
df.printSchema()

In [ ]:
import json
def parse_json(array_str):
json_obj = json.loads(array_str)
for item in json_obj:
   yield (item["a"], item["b"])

#### What are the column names?

In [ ]:
df.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

#### What does the Schema look like?

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



#### Print out the first 5 columns.

In [ ]:
# Didn't strictly need a for loop, could have just then head()
for row in df.head(5):
    print(row)
    print('\n')

Row(_c0='"{""tracking_date"":""2020-10-07""', _c1='""country_code"":""AR""', _c2='""seller_id"":""26606""', _c3='""seller_sk"":""AR|26606""', _c4='""counter2"":""1""}"')


Row(_c0='"{""tracking_date"":""2020-10-07""', _c1='""country_code"":""AR""', _c2='""seller_id"":""23899""', _c3='""seller_sk"":""AR|23899""', _c4='""counter2"":""1""}"')


Row(_c0='"{""tracking_date"":""2020-10-07""', _c1='""country_code"":""AR""', _c2='""seller_id"":""3178""', _c3='""seller_sk"":""AR|3178""', _c4='""counter2"":""1""}"')


Row(_c0='"{""tracking_date"":""2020-10-07""', _c1='""country_code"":""AR""', _c2='""seller_id"":""22247""', _c3='""seller_sk"":""AR|22247""', _c4='""counter2"":""1""}"')


Row(_c0='"{""tracking_date"":""2020-10-07""', _c1='""country_code"":""AR""', _c2='""seller_id"":""2924""', _c3='""seller_sk"":""AR|2924""', _c4='""counter2"":""1""}"')




#### Use describe() to learn about the DataFrame.

In [ ]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [ ]:
# Uh oh Strings! 
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
from pyspark.sql.functions import format_number
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519781|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+

